In [7]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [5]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

train = train.dropna()

In [2]:
# Initialize RFC and Vectorizer
rfc = RandomForestClassifier()
vect = TfidfVectorizer(stop_words='english')

In [3]:
# Define pipeline
pipe = Pipeline([('vect', vect), ('rfc', rfc)])

In [8]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500, 1000),
    'rfc__n_estimators': (20, 100, 400)
}

In [9]:
gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=4)

In [ ]:
gs.fit(train['description'], train['category'])

In [ ]:
gs.best_estimator_

In [ ]:
# Reinitialize RFC and Vectorizer
rfc = RandomForestClassifier()
vect = TfidfVectorizer(stop_words='english')

# Redfine pipeline (not sure if necessary)
pipe = Pipeline([('vect', vect), ('rfc', rfc)])

In [6]:
# Fit pipeline
pipe.fit(train['description'], train['category'])

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

# Making a Submission

In [28]:
# Test Pipeline
y_pred = pipe.predict(test['description'])

In [29]:
submission = pd.DataFrame({'id': test['id'], 'category':y_pred})
submission['category'] = submission['category'].astype('int64')

In [30]:
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [31]:
submission.to_csv('submission2.csv', index=False)

In [32]:
df = pd.read_csv('submission2.csv')
df.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1
